In [2]:
from transformers import T5Tokenizer
import pandas as pd

In [3]:
model_name = 'google/flan-t5-base'
train_file = 'hf://datasets/khairi/drug-discovery-hetionet/train.parquet'
valid_file = 'hf://datasets/khairi/drug-discovery-hetionet/validation.parquet'

In [4]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
train_df = pd.read_parquet(train_file)
valid_df = pd.read_parquet(valid_file)

In [6]:
train_df.head(2)

,input,target
0,Predict drugs for the central gene based on it...,- Isoniazid \n- Dapsone \n- Sulfamethoxazole \...
1,Predict drugs for the central gene based on it...,- Pentostatin \n- Adenosine \n- Edetic Acid \n...


In [24]:
def count_tokens(df):
    data = []
    for _, row in train_df.iterrows():
        data.append({
            'input_num_tokens': len(tokenizer.encode(row['input'])),
            'output_num_tokens': len(tokenizer.encode(row['target']))
        })
        
    return pd.DataFrame(data)

In [25]:
train_token_count = pd.DataFrame(count_tokens(train_df))
valid_token_count = pd.DataFrame(count_tokens(valid_df))

In [26]:
train_token_count.head(5)

,input_num_tokens,output_num_tokens
0,156,48
1,244,45
2,265,7
3,127,9
4,137,8


In [27]:
print('train dataset average input length: {} tokens'.format(train_token_count['input_num_tokens'].mean()))
print('train dataset max input length: {} tokens'.format(train_token_count['input_num_tokens'].max()))
print('train dataset min input length: {} tokens'.format(train_token_count['input_num_tokens'].min()))

train dataset average input length: 221.3681560730019 tokens
train dataset max input length: 270 tokens
train dataset min input length: 28 tokens


In [28]:
print('train dataset average output length: {} tokens'.format(train_token_count['output_num_tokens'].mean()))
print('train dataset max output length: {} tokens'.format(train_token_count['output_num_tokens'].max()))
print('train dataset min output length: {} tokens'.format(train_token_count['output_num_tokens'].min()))

train dataset average output length: 48.91818753933291 tokens
train dataset max output length: 3547 tokens
train dataset min output length: 5 tokens


In [33]:
print('average number of annotation per input: {}'.format(train_df['target'].map(lambda x: len(x.split('-'))).mean()))
print('maximum number of annotation: {}'.format(train_df['target'].map(lambda x: len(x.split('-'))).max()))
print('minimum number of annotation: {}'.format(train_df['target'].map(lambda x: len(x.split('-'))).min()))

average number of annotation per input: 8.07803650094399
maximum number of annotation: 517
minimum number of annotation: 2


In [35]:
train_df['num_annotation'] = train_df['target'].map(lambda x: len(x.split('-')))

train_df[train_df['num_annotation'] <= 10].shape, train_df.shape

((1403, 3), (1589, 3))

In [36]:
# TODO: filter out genes with more than 10 annotations (reduce output size) 